In [1]:
# Elasticsearch : Horizontally scalable search engine, 
# server process json and gives json
# text search, handle structured and aggregate data
# Often faster solution than Hadoop/Spark...

# documents - things you're searching for has ID and type (database)
# types - define schema and mapping shared by documents (table)
# indices - collection of types inverted indicies (collection of docs) (row)
from openai import OpenAI
from elasticsearch import Elasticsearch
import os

In [2]:
def llm(prompt):
    client = OpenAI(
        base_url='http://localhost:11434/v1/',
        api_key='ollama',
    )
    
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="phi3",
    )
    return response.choices[0].message.content

In [3]:
es_client = Elasticsearch('http://localhost:9200')

In [4]:
# import requests 

# docs_url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json'
# docs_response = requests.get(docs_url)
# documents_raw = docs_response.json()

# documents = []

# for course in documents_raw:
#     course_name = course['course']

#     for doc in course['documents']:
#         doc['course'] = course_name
#         documents.append(doc)

In [5]:
index_settings = {
    "settings": {
        "number_of_shards":1, # self contained instance 
        "number_of_replicas":0
    },
    "mappings": {
        "properties": {
            "text": {"type":"text"},
            "section": {"type":"text"},
            "question": {"type":"text"},
            "course": {"type":"keyword"}
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7e72852a2fe0>: Failed to establish a new connection: [Errno 111] Connection refused))

In [6]:
def elastic_search(query, index_name="course-questions"):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_doc = []

    for hit in response['hits']['hits']:
        result_doc.append(hit['_source'])
    return result_doc

In [7]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [8]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [9]:
rag("The course has already started, Can I still join in?")

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x708958e7bc40>: Failed to establish a new connection: [Errno 111] Connection refused))